# Example - The spam filter

### Introduction

In this example, we develop a **spam filter** based on a **decision tree**. A spam filter is a model which classifies e-mail messages as either spam or non-spam, using a collection of **numeric features** such as the frequency of certain words or characters. A differential trait of this example is that, in a spam filter, the **false positive rate**, that is, the proportion of non-spam messages wrongly classified as spam, must be very low.

### The data set

The file `spam.csv` contains data on 4,601 e-mail messages. Among these messages, 1,813 have been classified as spam. The data were gathered at Hewlett-Packard by merging: (a) a collection of spam e-mail from the company postmaster and the individuals who had filed spam, and (b) a collection of non-spam e-mail, extracted from filed work and personal e-mail.

The variables are:

* 48 numeric features whose names start with `word_`, followed by a word. They indicate the frequency, in percentage scale, with which that word appears in the message. Example: for a particular message, a value 0.21 for `word_make` means that 0.21% of the words in the message match the word 'make'.

* 3 numeric features indicating, respectively, the average length of uninterrupted sequences of capital letters (`cap_ave`), the length of the longest uninterrupted sequence of capital letters (`cap_long`) and the total number of capital letters in the message (`cap_total`).

* A dummy indicating whether that e-mail message is spam (`spam`).

Source: Hewlett-Packard. Taken from T Hastie, R Tibshirani & JH Friedman (2001), *The Elements of Statistical Learning*, Springer.

### Importing the data

As in other examples, I import the data, from a remote CSV file, to a **structured array**.

In [1]:
import numpy as np

In [2]:
path = 'https://raw.githubusercontent.com/cinnData/MLearning/main/Data/'
fname = path + 'spam.csv'
data = np.genfromtxt(fname, delimiter=',', names=True, dtype=None, encoding='utf-8')

`data` should be a structured array with 4,061 rows. Indeed:

In [3]:
data.shape

(4601,)

I check only the first row, because the source file has 52 columns:

In [4]:
data[:1]

array([(0., 0.64, 0.64, 0., 0.32, 0., 0., 0., 0., 0., 0., 0.64, 0., 0., 0., 0.32, 0., 1.29, 1.93, 0., 0.96, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 3.756, 61, 278, 1)],
      dtype=[('word_make', '<f8'), ('word_address', '<f8'), ('word_all', '<f8'), ('word_3d', '<f8'), ('word_our', '<f8'), ('word_over', '<f8'), ('word_remove', '<f8'), ('word_internet', '<f8'), ('word_order', '<f8'), ('word_mail', '<f8'), ('word_receive', '<f8'), ('word_will', '<f8'), ('word_people', '<f8'), ('word_report', '<f8'), ('word_addresses', '<f8'), ('word_free', '<f8'), ('word_business', '<f8'), ('word_email', '<f8'), ('word_you', '<f8'), ('word_credit', '<f8'), ('word_your', '<f8'), ('word_font', '<f8'), ('word_000', '<f8'), ('word_money', '<f8'), ('word_hp', '<f8'), ('word_hpl', '<f8'), ('word_george', '<f8'), ('word_650', '<f8'), ('word_lab', '<f8'), ('word_labs', '<f8'), ('word_telnet', '<f8'), ('word_857', '<f8'), ('word_data', '<f8'), ('

Everything looks right. I also check the **spam rate** in this data set:

In [5]:
round(np.mean(data['spam']), 3)

0.394

### Target vector and feature matrix

The **target vector** is the last column of the data set:

In [6]:
y = data['spam']

For the **feature matrix**, I convert the resulting subarray to an **unstructured array**, leaving aside the last column:

In [7]:
X = data[list(data.dtype.names[:-1])]
from numpy.lib.recfunctions import structured_to_unstructured
X = structured_to_unstructured(X)

The shape of this matrix is as expected:

In [8]:
X.shape

(4601, 51)

### Decision tree classifier

To develop a tree decision classifier, I use the **estimator class** `DecisionTreeClassifier` from the scikit-learn subpackage `tree`. This is imported as usual:

In [9]:
from sklearn.tree import DecisionTreeClassifier

I instantiate a first estimator from this class. I set `max_depth=2`, which limits the **depth**, that is, the length of the longest branch.

In [10]:
treeclf1 = DecisionTreeClassifier(max_depth=2)

The method `fit` finds the optimal tree under this specification. This tree can be seen at `github.com/cinnData/MLearning/blob/main/7.%20Decision%20trees/fig 7.2.png`. I has four leaves and uses only three of the 51 features available.

In [11]:
treeclf1.fit(X, y)

DecisionTreeClassifier(max_depth=2)

The method `score` gives us the **accuracy** of this model:

In [12]:
round(treeclf1.score(X, y), 3)

0.834

Though this is a not case of **class imbalance**, the accuracy is not appropriate for the evaluation of this model, by different reasons. Let me postpone the discussion to the end of the exmaple. 

### Confusion matrix

Let us take a look at the **confusion matrix**. First, I obtain the predicted target values with the method `predict`. 

In [13]:
ypred1 = treeclf1.predict(X)

The confusion matrix resulting from cross tabulating `y` and `ypred1`. It can be obtained by means of the method `confusion_matrix`, from the subpackage `metrics`:

In [14]:
from sklearn.metrics import confusion_matrix
conf1 = confusion_matrix(y, ypred1)
conf1

array([[2575,  213],
       [ 549, 1264]], dtype=int64)

The **true positive rate** and the **false positive rate** can be extracted from this matrix (also calculated directly):

In [15]:
tp1 = conf1[1, 1]/np.sum(conf1[1, :])
fp1 = conf1[0, 1]/np.sum(conf1[0, :])
round(tp1, 3), round(fp1, 3)

(0.697, 0.076)

The weakest point of this model is the false positive rate, which is a bit too high. But it is a small tree, so let us see whether further branching can improve these statistics.

### A deeper tree

I try next a tree with `max_depth=3`.

In [16]:
treeclf2 = DecisionTreeClassifier(max_depth=3)
treeclf2.fit(X, y)
round(treeclf2.score(X, y), 3)

0.87

The accuracy has clearly improved. The new confusion matrix is:

In [17]:
ypred2 = treeclf2.predict(X)
conf2 = confusion_matrix(y, ypred2)
conf2

array([[2598,  190],
       [ 406, 1407]], dtype=int64)

The true positive and false positive rates both improve, though the second one is still a bit high.

In [18]:
tp2 = conf2[1, 1]/np.sum(conf2[1, :])
fp2 = conf2[0, 1]/np.sum(conf2[0, :])
tp2.round(3), fp2.round(3)

(0.776, 0.068)

### More depth

I set now `max_depth=4`:

In [19]:
treeclf3 = DecisionTreeClassifier(max_depth=4)
treeclf3.fit(X, y)
round(treeclf3.score(X, y), 3)

0.891

The accuracy is still improving. Let us take a look at the confusion matrix:

In [20]:
ypred3 = treeclf3.predict(X)
conf3 = confusion_matrix(y, ypred3)
conf3

array([[2627,  161],
       [ 341, 1472]], dtype=int64)

In [21]:
tp3 = conf3[1, 1]/np.sum(conf3[1, :])
fp3 = conf3[0, 1]/np.sum(conf3[0, :])
round(tp3, 3), round(fp3, 3)

(0.812, 0.058)

Let me close the example with a final model, based on a deeper tree.

### A final model

I set now `max_depth=5`:

In [22]:
treeclf4 = DecisionTreeClassifier(max_depth=5)
treeclf4.fit(X, y)
round(treeclf4.score(X, y), 3)

0.904

The accuracy still improves a bit. The confusion matrix is, now:

In [23]:
ypred4 = treeclf4.predict(X)
conf4 = confusion_matrix(y, ypred4)
conf4

array([[2696,   92],
       [ 350, 1463]], dtype=int64)

We find an interesting decrease in the number of false positives, which is relevant for his example. The false positive rate confirms this:

In [24]:
tp4 = conf4[1, 1]/np.sum(conf4[1, :])
fp4 = conf4[0, 1]/np.sum(conf4[0, :])
round(tp4, 3), round(fp4, 3)

(0.807, 0.033)

Although I got stuck with the positive rate, the false positive rate is promising.

### Feature importance

One of the most attractive traits of decision trees is that they produce, as a by-product, a ranking of the features by their contribution to the predictive power of the model. This is extracted by the method `feature_importances_`, which returns a 1d array in which every term is the **importance** of one of the features, measured as the **percentage of reduction of the impurity** due to the splits in which the feature is involved. A null value indicates that the corresponding feature is not involved in any split, so it is not used by the decision tree.

In [25]:
imp = treeclf4.feature_importances_

So our biggest tree uses 15 features, out from 51. This is not surprising, since this is maximum number of features that a tree obtained under the specification `max_depth=4` can use. These features can be listed in:

In [26]:
feat_list = np.array(data.dtype.names[:-1])[imp > 0]

In [27]:
feat_imp = np.round(treeclf4.feature_importances_[imp > 0], 3)

A report can be extracted with NumPy as:

In [28]:
feat_report = np.array([feat_list, feat_imp], dtype=object).transpose()
feat_report[np.argsort(np.array(feat_report[:, 1], dtype='float'))[::-1], :]

array([['word_remove', 0.406],
       ['word_free', 0.233],
       ['word_money', 0.108],
       ['word_000', 0.057],
       ['cap_ave', 0.052],
       ['word_george', 0.041],
       ['word_hp', 0.039],
       ['word_edu', 0.033],
       ['word_internet', 0.009],
       ['cap_long', 0.005],
       ['word_mail', 0.005],
       ['word_our', 0.004],
       ['word_650', 0.003],
       ['word_hpl', 0.003],
       ['cap_total', 0.002],
       ['word_technology', 0.001]], dtype=object)

Don't pay too much attention at the trick which I applied to extract the list sorted by the importance. These are NumPy technicalities.

### Final discussion

* This example shows how the predictions can be improved by increasing the **model complexity**. In the decision trees, the complexity is visualized as the tree getting more branches, so producing a finer partition of the data set into leaf nodes.

* Even if you do not see the improvement halting at a certain degree of complexity, you must be cautious, because, in machine learning, complexity comes hand in hand with **overfitting**: when our model becomes too well adapted to the training data, it does not achieve the same performance with fresh data, not used to develop the model. How to deal with overfitting will be seen later in this course.

* The data set of this example is not a "representative sample of a population". The spam rate is not the one Hewlett-Packard was finding in real operations, but an artificial one. So you must be careful when extrapolating the results obtained. Any calculation involving the two columns of the confusion matrix, that is, mixing spam with non-spam, is suspicious. The accuracy is an example. The true accuracy should be estimated as the weighted average of the accuracy in the spam subpopulation and the accuracy in the non-spam subpopulation. But we don't know how to weight these two things. 

* On the other hand, as far as we can reproduce them in some **test data**, the true positive and false positive rates can be trusted, since they are calculated separately, in the spam and the non-spam part, respectively. 

### Homework

1. Drop the three `cap_` variables and **binarize** all the `word_` variables, transforming them into dummies for the occurrence of the corresponding word. Develop two spam classifiers, one based on a **logistic regression** equation and the other one based on a **decision tree**, using the binarized data set.

2. Evaluate these classifiers based on their respective **confusion matrices**.